In [94]:
import re
import sys
import os
# !pip3 install nest_asyncio
# !pip3 install twint
# !pip3 install nltk
import nest_asyncio
import string
nest_asyncio.apply()
import twint
import pandas as pd
import nltk
import textblob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize, RegexpTokenizer
from nltk.tokenize import sent_tokenize, PunktSentenceTokenizer, WhitespaceTokenizer
import preprocessor as p
from textblob import TextBlob
# nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dominic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [95]:
candidates = ["michael bennet", "joseph biden",
              "michael bloomberg", "cory booker",
              "steve bullock", "pete buttigieg",
              "julian castro", "john delaney",
              "tulsi gabbard", "kamala harris",
              "amy klobuchar", "deval patrick",
              "bernie sanders", "joe sestak",
              "tom steyer", "elizabeth warren",
              "marianne williamson", "andrew yang",
              "donald trump", "joe walsh"]


class candidate:
    def __init__(self, name):
        self.name = name
        self.tweets = pd.DataFrame()
    def tweet_crawler(self, count=10):
        api = TwitterClient()
        self.tweets = api.get_tweets(self.name, count)



# class Preprocess:
#     def iterate(self):
#         for preprocess_method in [self.remove_urls,
#                                   self.remove_usernames,
#                                   self.remove_numbers,
#                                   self.remove_special_chars,
#                                   self.remove_non_ascii,
#                                   self.remove_stopwords]:
#             yield preprocess_method

#     @staticmethod
#     def remove_by_regex(tweets, regexp):
#         # print(tweets)
#         print(tweets.loc[:, "tweet"])
#         if tweets is not None:
#             tweets.loc[:, "tweet"].sub(regexp, "", tweets.loc[:, "tweet"])
#         # print(tweets)
#         # tweets = regexp.sub('', tweets)
#         # tweets.replace("", regexp, inplace=True)

#     def remove_urls(self, tweets):
#         return Preprocess.remove_by_regex(tweets, re.compile(r"http.?://[^\s]+[\s]?"))

#     def remove_usernames(self, tweets):
#         return Preprocess.remove_by_regex(tweets, re.compile(r"@[^\s]+[\s]?"))

#     def remove_numbers(self, tweets):
#         return Preprocess.remove_by_regex(tweets, re.compile(r"\s?[0-9]+\.?[0-9]*"))

#     def remove_special_chars(self, tweets):  # it unrolls the hashtags to normal words
#         for remove in map(lambda r: regex.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
#                                                                      "@", "%", "^", "*", "(", ")", "{", "}",
#                                                                      "[", "]", "|", "/", "\\", ">", "<", "-",
#                                                                      "!", "?", ".", "'",
#                                                                      "--", "---", "#"]):
#             tweets.loc[:, "tweet"].replace(remove, "", inplace=True)
#         return tweets

#     def remove_non_ascii(self, tweets):
#         return Preprocess.remove_by_regex(tweets, regex.compile(r"[^\x00-\x7F]+"))

#     def remove_stopwords(self, tweets):
#         stop_words = set(stopwords.words('english'))
#         word_tokens = word_tokenize(tweet)
#         processed_tweet = [word for word in word_tokens if not word in stop_words]
#         return ' '.join(processed_tweet)


class TwitterClient:
    '''
    Twitter Twint intialization
    '''
    def __init__(self):
        #Intializes the Twint object
        self.twint_Api = twint.Config()

    def clean_tweet(self, tweet):
        # tweet = p.clean(tweet['text'])
        # Check characters to see if they are in punctuation
        nopunc = [char for char in tweet if char not in string.punctuation]
        # Join the characters again to form the string.
        nopunc = ''.join(nopunc)
        # convert text to lower-case
        nopunc = nopunc.lower()
        # remove URLs
        nopunc = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', nopunc)
        nopunc = re.sub(r'http\S+', '', nopunc)
        # remove usernames
        nopunc = re.sub('@[^\s]+', '', nopunc)
        # remove the # in #hashtag
        nopunc = re.sub(r'#([^\s]+)', r'\1', nopunc)
        # remove repeated characters
        nopunc = word_tokenize(nopunc)
        # remove stopwords from final word list
        words = [word for word in nopunc if word not in stopwords.words('english')]
        return " ".join(words)
    
    def preprocess(self, data ):
        cleaned_tweets = {}
        newdata = []

        stop_words = stopwords.words('english')
        words = set(nltk.corpus.words.words())

        for token in data.tweet: 
            newdoc = ""
            #print(token)
            seperator = " " 
            token = re.sub('[^A-Za-z0-9]+', ' ', token) #remove special characters
            token = re.sub(r'\d+', '', token) #remove numbers
            #token = re.sub(r'\b\w{1,2}\b', '', token) #remove words with <= 2 characters
            token = " ".join(w for w in nltk.wordpunct_tokenize(token) \
                    # if w.lower() in words or 
                     or not w.isalpha())
            whitespace_token = WhitespaceTokenizer().tokenize( token )
            #wo_stopwords_token = [x for x in whitespace_token
            #                      if not x in stop_words]
            newdoc = seperator.join( (whitespace_token) ).lower()
            #print(newdoc)
            newdata.append(newdoc)
        
        print(newdata)
        cleaned_tweets['tweet'] = newdata
    
        newtweets = pd.DataFrame(cleaned_tweets)

        return newtweets

    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        return analysis.sentiment.polarity, analysis.sentiment.subjectivity

    def twint_to_pandas(self, col):
        return twint.output.panda.Tweets_df[col]

    def get_tweets(self, search, count):

        #Creates a search string from the list of presidential candidates
        self.twint_Api.Search = search
        self.twint_Api.Limit = count
        self.twint_Api.Format = "Username: {username} |  Tweet: {tweet}  |  Location: {location}"
        self.twint_Api.Lang = "en"
        self.twint_Api.Pandas = True
        self.Output = "output.csv"
        # self.twint_Api.Pandas_au = True
        self.twint_Api.Location = True
        self.twint_Api.Hide_output = True

        twint.run.Search(self.twint_Api)

        tweets_df = self.twint_to_pandas(["date", "username", "tweet", "hashtags"])
        #cleaned_tweets = preprocess( tweets_df )
        
        #print(tweets_df.keys())
        tweet_sentiment = []
        tweet_subjectivity = []
        
        tweets_df = self.preprocess(tweets_df)
        print(tweets_df)
        # self.clean_tweet(tweets_df)
        for tweet in tweets_df.tweet:
            #print(tweet)
            sentiment = self.get_tweet_sentiment(tweet)
            tweet_sentiment.append(sentiment[0])
            tweet_subjectivity.append(sentiment[1])
            #print(tweet)
            
        print(tweet_sentiment)
        tweets_df['Sentiment'] = tweet_sentiment
        return tweets_df

In [96]:
test = candidate(candidates[2])
test.tweet_crawler()

['not that i am aware of your point', 'mikebloomberg another wannabe president that is willing to bend over for the money goawaybloomberg pic twitter com exxfgtxovx', 'bloomberg is authoritarian af if it s him vs trump i m sitting it out and going back to not giving a shit we ll deserve what we get at that point', 'hadn t even thought of that but it is their money no matter how foolishly spent', 'much as i find many of your statements annoying this one is accurate well not the race and gender parts zero proof of that here but money absolutely bloomberg s in our system directly lets him continue while she can t', 'those of you that are supporting mikebloomberg which side of the political isle do you typically lean bloomberg election', 'hey mike can i have mile of your mile high stack of bills', 'go away', 'you realize that he will never get the black hispanic vote right', 'new show trump fakes orgasm at rally michael bloomberg polling at mark zuckerberg s dinner with trump and much more

In [97]:
print(test.tweets.keys())

Index(['tweet', 'Sentiment'], dtype='object')
